# **CodeBERT Experiments**

1. [x] load CodeBERT model
2. [x] run inference / get embeddings
3. [x] inverse embeddings
4. [x] edit embeddings
5. [x] inverse edited embeddings

## References

* https://huggingface.co/microsoft/codebert-base-mlm

## Environment

In [1]:
!python --version

Python 3.7.10


In [2]:
!pip install tokenizers
!pip install transformers

     |████████████████████████████████| 3.2MB 16.1MB/s 
     |████████████████████████████████| 2.0MB 16.9MB/s 
     |████████████████████████████████| 890kB 37.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=9d1fc1d995c404ede42cf9e953abe7c96dd2cde96fc7ee1d6e894059f2c61e07
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


## Dependencies

In [3]:
import torch
import numpy as np
from scipy.special import softmax
from transformers import pipeline
from transformers import RobertaTokenizer, RobertaForMaskedLM

## Settings

In [6]:
model_name = 'microsoft/codebert-base-mlm'

## 1. Load CodeBERT model

In [7]:
%time model = RobertaForMaskedLM.from_pretrained(model_name)
%time tokenizer = RobertaTokenizer.from_pretrained(model_name)

%time fill_mask = pipeline('fill-mask', model=model, tokenizer=tokenizer)


CPU times: user 14.1 s, sys: 1.97 s, total: 16.1 s
Wall time: 16.8 s



CPU times: user 433 ms, sys: 44.7 ms, total: 477 ms
Wall time: 738 ms
CPU times: user 409 µs, sys: 0 ns, total: 409 µs
Wall time: 414 µs


## 2. Run inference

In [8]:
torch.set_grad_enabled(False)

In [9]:
text1 = 'x = a + b'
text2 = 'x = a - b'
texts = [text1, text2]

In [10]:
tokens = {}

for text in texts:
  tokens_pt = tokenizer(text, return_tensors='pt')
  tokens[text] = tokens_pt

for text, tokens_pt in tokens.items():
  print(f'text: >{text}<')
  for key, value in tokens_pt.items():
    print(f'\t{key}: {value}')
    if key == 'input_ids':
      print(f'\t\ttokens (str): {[tokenizer.convert_ids_to_tokens(s) for s in value]}')
      print(f'\t\t#decoding: {[tokenizer.decode(v) for v in value]}')

text: >x = a + b<
	input_ids: tensor([[   0, 1178, 5457,   10, 2055,  741,    2]])
		tokens (str): [['<s>', 'x', 'Ġ=', 'Ġa', 'Ġ+', 'Ġb', '</s>']]
		#decoding: ['<s>x = a + b</s>']
	attention_mask: tensor([[1, 1, 1, 1, 1, 1, 1]])
text: >x = a - b<
	input_ids: tensor([[   0, 1178, 5457,   10,  111,  741,    2]])
		tokens (str): [['<s>', 'x', 'Ġ=', 'Ġa', 'Ġ-', 'Ġb', '</s>']]
		#decoding: ['<s>x = a - b</s>']
	attention_mask: tensor([[1, 1, 1, 1, 1, 1, 1]])


In [13]:
embeddings = {}
for text, tokens_pt in tokens.items():
  %time output = fill_mask.model.roberta(**tokens_pt)

  last_hidden_state = output.last_hidden_state

  embeddings[text] = last_hidden_state

CPU times: user 93 ms, sys: 1.08 ms, total: 94 ms
Wall time: 202 ms
CPU times: user 74.1 ms, sys: 0 ns, total: 74.1 ms
Wall time: 73.7 ms


## 4. Inverse embeddings

In [22]:
for text, embedding in embeddings.items():
  print(text)
  lm_head_output = fill_mask.model.lm_head(embedding)

  for i in range(lm_head_output.shape[1]):
    probs = softmax(lm_head_output[0][i].detach().cpu().numpy())
    indices = probs.argsort()[-10:][::-1]
    print([(tokenizer.decode(int(idx)), round(probs[idx], 3)) for idx in indices])

x = a + b
[('\n', 0.738), ('x', 0.022), (' )', 0.016), (' }', 0.016), ('.', 0.012), (' ]', 0.007), ('.', 0.007), ('_', 0.006), (' "', 0.006), (" '", 0.005)]
[('x', 0.994), ('y', 0.004), (' x', 0.0), ('z', 0.0), ('b', 0.0), ('xy', 0.0), ('xx', 0.0), ('xc', 0.0), (' y', 0.0), ('X', 0.0)]
[(' =', 0.999), (' +=', 0.0), (' +', 0.0), (' (', 0.0), (' -', 0.0), ('=', 0.0), (' -=', 0.0), (' :', 0.0), (',', 0.0), (' >', 0.0)]
[(' a', 1.0), (' b', 0.0), (' 1', 0.0), ('a', 0.0), (' ax', 0.0), (' A', 0.0), (' 0', 0.0), (' c', 0.0), (' h', 0.0), (' e', 0.0)]
[(' +', 1.0), (' -', 0.0), (' *', 0.0), ('+', 0.0), (' plus', 0.0), (' /', 0.0), (' +=', 0.0), (' ^', 0.0), (' x', 0.0), (' =', 0.0)]
[(' b', 1.0), (' a', 0.0), (' B', 0.0), (' 1', 0.0), (' 2', 0.0), (' f', 0.0), ('b', 0.0), (' ab', 0.0), (' y', 0.0), (' c', 0.0)]
[(' b', 0.123), (' p', 0.054), (' y', 0.052), (' +', 0.043), (' s', 0.027), (' -', 0.027), (' #', 0.024), (' m', 0.023), (' x', 0.021), (' c', 0.02)]
x = a - b
[('\n', 0.537), ('x', 0.

## 5. Edit embeddings

In [19]:
plus_embeddings = embeddings[text1]
minus_embeddings = embeddings[text2]

new_embeddings = 0.5*(plus_embeddings + minus_embeddings)

## 6. Inverse edited embeddings

In [23]:
lm_head_output = fill_mask.model.lm_head(new_embeddings)

for i in range(lm_head_output.shape[1]):
  probs = softmax(lm_head_output[0][i].detach().cpu().numpy())
  indices = probs.argsort()[-10:][::-1]
  print([(tokenizer.decode(int(idx)), round(probs[idx], 3)) for idx in indices])

[('\n', 0.648), ('x', 0.032), (' )', 0.022), ('.', 0.018), (' }', 0.014), (' ]', 0.013), ('_', 0.008), ('.', 0.008), (' :', 0.008), (' <', 0.007)]
[('x', 0.995), ('y', 0.003), (' x', 0.001), ('z', 0.0), ('b', 0.0), ('xy', 0.0), ('xx', 0.0), ('xc', 0.0), (' y', 0.0), ('index', 0.0)]
[(' =', 0.998), (' +', 0.0), (' -', 0.0), (' (', 0.0), (' +=', 0.0), ('=', 0.0), (' :', 0.0), (' >', 0.0), (' -=', 0.0), (' <', 0.0)]
[(' a', 1.0), (' b', 0.0), (' 1', 0.0), (' ax', 0.0), (' 0', 0.0), (' c', 0.0), ('a', 0.0), (' A', 0.0), (' x', 0.0), (' e', 0.0)]
[(' +', 0.899), (' -', 0.1), (' *', 0.0), (' ^', 0.0), (' /', 0.0), (' x', 0.0), (' >', 0.0), (' <', 0.0), (' :', 0.0), ('.', 0.0)]
[(' b', 1.0), (' a', 0.0), (' B', 0.0), (' 2', 0.0), (' 1', 0.0), (' f', 0.0), ('b', 0.0), (' ab', 0.0), (' y', 0.0), (' c', 0.0)]
[(' b', 0.157), (' -', 0.061), (' +', 0.058), (' #', 0.045), (' y', 0.045), (' x', 0.033), (' p', 0.032), (' s', 0.023), (' c', 0.023), (' ', 0.022)]
